### Bài 1:

In [1]:
from transformers import pipeline

# SỬA Ở ĐÂY: Thêm tham số model="bert-base-uncased" hoặc "distilbert-base-uncased"
mask_filler = pipeline("fill-mask", model="bert-base-uncased")

# Giữ nguyên [MASK] đúng chuẩn BERT
input_sentence = "Hanoi is the [MASK] of Vietnam."

predictions = mask_filler(input_sentence, top_k=5)

print(f"Câu gốc: {input_sentence}")
for pred in predictions:
    print(f"Dự đoán: '{pred['token_str']}' -> Câu: {pred['sequence']}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if 

Câu gốc: Hanoi is the [MASK] of Vietnam.
Dự đoán: 'capital' -> Câu: hanoi is the capital of vietnam.
Dự đoán: 'center' -> Câu: hanoi is the center of vietnam.
Dự đoán: 'birthplace' -> Câu: hanoi is the birthplace of vietnam.
Dự đoán: 'headquarters' -> Câu: hanoi is the headquarters of vietnam.
Dự đoán: 'city' -> Câu: hanoi is the city of vietnam.


#### Mô hình bert-base-uncased đã dự đoán từ "capital" với độ tin cậy cao nhất (xếp vị trí đầu tiên trong danh sách kết quả).
#### Lý do vì sao các mô hình Encoder-only như BERT lại phù hợp cho tác vụ này:

* Mô hình Encoder-only (như BERT) sử dụng cơ chế Attention hai chiều (Bidirectional Attention).

* Để điền đúng từ vào [MASK], mô hình cần nhìn thấy cả ngữ cảnh bên trái ("Hanoi is the...") và bên phải ("...of Vietnam").

* Kiến trúc Encoder cho phép mô hình "đọc" toàn bộ câu cùng một lúc để hiểu trọn vẹn ngữ nghĩa, từ đó đưa ra dự đoán chính xác nhất. Nếu chỉ đọc một chiều (từ trái sang phải), mô hình có thể gặp khó khăn vì chưa biết đối tượng phía sau là "Vietnam".

### Bài 2:

In [2]:
from transformers import pipeline, set_seed

# Đặt seed để kết quả có thể tái lập (tùy chọn)
set_seed(42)

# 1. Tải pipeline "text-generation" (thường dùng GPT-2)
generator = pipeline("text-generation", model="gpt2")

# 2. Đoạn văn bản mồi
prompt = "The best thing about learning NLP is"

# 3. Sinh văn bản
# pad_token_id được gán bằng eos_token_id để tránh cảnh báo trong log
generated_texts = generator(prompt, max_length=50, num_return_sequences=1, pad_token_id=50256)

# 4. In kết quả
print(f"Câu mồi: '{prompt}'")
for text in generated_texts:
    print("\nVăn bản được sinh ra:")
    print(text['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Câu mồi: 'The best thing about learning NLP is'

Văn bản được sinh ra:
The best thing about learning NLP is that you learn to learn something, and you learn it through hard work and hard work. It's not like you learn all at once, but you learn at what is most important.

If you want to learn more, there are a lot of books out there that are written about NLP, and I think that's why they make sense. But at the same time, there are many books that have been written about NLP that people have done that have actually been successful.

JUAN GONZÁLEZ: Well, this is a little bit of an aside. On a personal note, I'm really curious to know what the people who have been on the frontlines of NLP are doing right now. I know there's an organization called the NLP Project, and they're making a lot of changes going forward, and I'm curious about how they're doing with the NLP Project.

So, the NLP Project is a group of people who are doing about two things right now. One is trying to get the word out

#### Kết quả sinh ra có hợp lý không?

* Về mặt ngữ pháp: Kết quả rất trôi chảy và đúng ngữ pháp tiếng Anh (VD: "The best thing about learning NLP is that you learn to learn something...").

* Về mặt ngữ nghĩa:

    * Phần đầu khá hợp lý và liên quan đến chủ đề học tập/sách vở.

    * Phần sau bị lạc đề (hallucination). Mô hình tự động chuyển sang định dạng kịch bản phỏng vấn với nhân vật giả định ("JUAN GONZÁLEZ") và một tổ chức không rõ ràng ("NLP Project").

##### Kết luận: Mô hình tốt ở việc tạo câu đúng cú pháp nhưng thiếu khả năng duy trì mạch tư duy logic dài

#### Vì sao các mô hình Decoder-only như GPT lại phù hợp cho tác vụ này?

* Mô hình Decoder-only hoạt động theo cơ chế Tự hồi quy (Autoregressive).

* Nó sử dụng Causal Masking (Che giấu nhân quả), tức là tại thời điểm dự đoán từ hiện tại, nó chỉ được phép nhìn thấy các từ trong quá khứ (bên trái) và bị che hoàn toàn các từ tương lai (bên phải).

* Cơ chế này mô phỏng chính xác cách con người viết văn bản tự nhiên (viết từng từ nối tiếp nhau từ trái sang phải), do đó nó tối ưu cho tác vụ sinh văn bản (Text Generation/Next Token Prediction).

### Bài 3:

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel

# 1. Chọn một mô hình BERT
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. Câu đầu vào
sentences = ["This is a sample sentence."]

# 3. Tokenize câu
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# 4. Đưa qua mô hình để lấy hidden states
with torch.no_grad():
    outputs = model(**inputs)

# last_hidden_state chứa vector đầu ra của tất cả các token
# Kích thước: (batch_size, sequence_length, hidden_size)
last_hidden_state = outputs.last_hidden_state

# 5. Thực hiện Mean Pooling (Tính trung bình cộng có trọng số của mask)
attention_mask = inputs['attention_mask']

# Mở rộng mask để khớp kích thước với last_hidden_state
mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()

# Tính tổng các vector (chỉ những token thật, bỏ qua padding)
sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)

# Tính tổng số lượng token thật (tránh chia cho 0 bằng cách kẹp min=1e-9)
sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)

# Chia tổng cho số lượng để ra trung bình
sentence_embedding = sum_embeddings / sum_mask

# 6. In kết quả
print("Vector biểu diễn của câu (5 giá trị đầu):")
print(sentence_embedding[0][:5]) # Chỉ in 5 số đầu để gọn
print("\nKích thước của vector:", sentence_embedding.shape)

Vector biểu diễn của câu (5 giá trị đầu):
tensor([-0.0639, -0.4284, -0.0668, -0.3843, -0.0658])

Kích thước của vector: torch.Size([1, 768])


#### Kích thước của vector biểu diễn là 768. Con số này tương ứng với tham số hidden_size (kích thước lớp ẩn) của kiến trúc bert-base.

#### Lý do cần sử dụng attention_mask khi thực hiện Mean Pooling:

* Để đảm bảo tính trung bình cộng phản ánh đúng nội dung câu, chúng ta cần loại bỏ các token đệm (padding tokens).

* Trong xử lý theo batch, các câu ngắn được thêm các số 0 (padding) để bằng độ dài câu dài nhất. Nếu tính trung bình cộng tất cả (bao gồm cả các số 0 này), giá trị vector sẽ bị chia nhỏ đi và làm lệch lạc ngữ nghĩa của câu.

* attention_mask giúp đánh dấu đâu là từ thật (giá trị 1), đâu là đệm (giá trị 0). Phép toán sum_embeddings / sum_mask đảm bảo ta chỉ chia tổng vector cho số lượng từ thực tế có trong câu.